In [4]:
import pandas as pd
from typing import Dict
import warnings

# Ignorar avisos para uma apresentação mais limpa
warnings.filterwarnings('ignore')

df_audit_log = pd.read_csv('../datasets/admin_auditlog.csv')
df_apps = pd.read_csv('../datasets/admin_apps.csv')
df_ambientes = pd.read_csv('../datasets/admin_environments.csv')

print(f"Arquivo de log de auditoria carregado com sucesso!")
print(f"- Total de registros de log: {df_audit_log.shape[0]}")
print(f"- Total de registros de aplicativos: {df_apps.shape[0]}")
print(f"- Total de registros de ambientes: {df_ambientes.shape[0]}")

print(f"Colunas do DataFrame de log de auditoria: {df_audit_log.columns.tolist()}")
print(f"Colunas do DataFrame de aplicativos: {df_apps.columns.tolist()}")
print(f"Colunas do DataFrame de ambientes: {df_ambientes.columns.tolist()}")

print("\nIniciando limpeza e conversão de colunas de data...")

# Limpando a tabela de aplicativos
df_apps['admin_appcreatedon'] = pd.to_datetime(df_apps['admin_appcreatedon'], errors='coerce')
df_apps['admin_applastlaunchedon'] = pd.to_datetime(df_apps['admin_applastlaunchedon'], errors='coerce')

# Limpando a tabela de log de auditoria
df_audit_log['Creation Time'] = pd.to_datetime(df_audit_log['Creation Time'], errors='coerce')

# Limpando a tabela de ambientes
df_ambientes['admin_environmentcreatedon'] = pd.to_datetime(df_ambientes['admin_environmentcreatedon'], errors='coerce')


print("Colunas de data tratadas com sucesso!")
#----------------------------------------------------


Arquivo de log de auditoria carregado com sucesso!
- Total de registros de log: 582220
- Total de registros de aplicativos: 4036
- Total de registros de ambientes: 336
Colunas do DataFrame de log de auditoria: ['Audit Log', 'Created On', 'createdby', 'Modified On', 'modifiedby', 'createdonbehalfby', 'modifiedonbehalfby', 'Created By', 'createdbyyominame', 'Created By (Delegate)', 'createdonbehalfbyyominame', 'Modified By', 'modifiedbyyominame', 'Modified By (Delegate)', 'modifiedonbehalfbyyominame', 'ownerid', 'Owner', 'owneridyominame', 'owningbusinessunit', 'owninguser', 'owningteam', 'Owning Business Unit', 'Status', 'statecodename', 'Status Reason', 'statuscodename', 'Import Sequence Number', 'Record Created On', 'Time Zone Rule Version Number', 'UTC Conversion Time Zone Code', 'Title', 'App ID', 'admin_applookup', 'Creation Time', 'Operation', 'admin_powerappsapplogs', 'User City', 'User Country', 'User Department', 'User Job Title', 'User UPN', 'Workload', 'PowerApps App Logs', '

In [6]:
coluna_app_id_log = 'App ID'
coluna_usuario_log = 'User UPN'
coluna_data_log = 'Creation Time'
coluna_evento_log = 'Operation' # Usaremos esta para contar o total de sessões/eventos

print("Calculando métricas de uso a partir do log de auditoria...")

# Converte a coluna de data para o formato datetime para garantir a ordenação correta
df_audit_log[coluna_data_log] = pd.to_datetime(df_audit_log[coluna_data_log])

# Agrupa os dados por App ID e calcula as métricas
metricas_de_uso = df_audit_log .groupby(coluna_app_id_log).agg(
    total_sessoes=(coluna_evento_log, 'size'),          # Conta o número de eventos (sessões)
    usuarios_unicos=(coluna_usuario_log, 'nunique'),   # Conta o número de usuários distintos
    ultimo_acesso=(coluna_data_log, 'max')             # Pega a data mais recente
).reset_index()

print("Métricas calculadas com sucesso!")
print("\nAmostra da nova tabela 'metricas_de_uso':")
display(metricas_de_uso.head())

Calculando métricas de uso a partir do log de auditoria...
Métricas calculadas com sucesso!

Amostra da nova tabela 'metricas_de_uso':


,App ID,total_sessoes,usuarios_unicos,ultimo_acesso
0,007e478b-dda9-4f1b-b7c1-36532b0d9201,12,3,2025-06-16 15:03:31
1,01460fa8-6f1c-4d98-bc22-ff2e184c2c45,1,1,2025-05-05 12:31:27
2,01d39f7e-ec1f-40a0-91f3-d94e1eb18fa0,1,1,2025-01-28 13:25:57
3,02426944-7c8d-41cf-a010-45bb3f316cd9,7045,187,2025-06-25 12:05:18
4,02442824-b7f8-49d7-ae5f-fde3adefde5c,12,1,2025-06-20 19:40:31


In [7]:
# Célula para mesclar as métricas de uso E os nomes dos ambientes

print("Iniciando a junção de todos os dados...")

# --- 1. Junção com as Métricas de Uso ---
df_apps_com_metricas = pd.merge(
    df_apps,
    metricas_de_uso,
    left_on='admin_appid',
    right_on='App ID',
    how='left'
)
df_apps_com_metricas['total_sessoes'].fillna(0, inplace=True)
df_apps_com_metricas['usuarios_unicos'].fillna(0, inplace=True)
df_apps_com_metricas['total_sessoes'] = df_apps_com_metricas['total_sessoes'].astype(int)
df_apps_com_metricas['usuarios_unicos'] = df_apps_com_metricas['usuarios_unicos'].astype(int)

# --- 2. Junção com a Tabela de Ambientes ---
print("Adicionando o nome do Ambiente a cada aplicativo...")

df_apps_com_tudo = pd.merge(
    df_apps_com_metricas,
    df_ambientes[['admin_environmentid', 'admin_displayname']],
    left_on='admin_appenvironmentid',
    right_on='admin_environmentid',
    how='left',
    suffixes=('_app', '_ambiente') # Adiciona sufixos para colunas com mesmo nome
)

print("✅ Junção completa!")
print("\nAmostra da tabela enriquecida com o Nome do Ambiente:")
display(df_apps_com_tudo[['admin_displayname_app', 'admin_displayname_ambiente', 'usuarios_unicos']].head())

Iniciando a junção de todos os dados...
Adicionando o nome do Ambiente a cada aplicativo...
✅ Junção completa!

Amostra da tabela enriquecida com o Nome do Ambiente:


,admin_displayname_app,admin_displayname_ambiente,usuarios_unicos
0,[Licença] - Gerenciamento de Licenças,PRD_IDCER_SOLUTIONS,721
1,Resultado da Avaliação de Integridade (RAI),PRD_RAI_GESTAO_REL_INTEGRIDADE,9
2,Chamados - Estudos Econômicos,NaN,63
3,Gestão de Avaliações da BRMED,NaN,16
4,Chamados - FRPCA,NaN,238


In [25]:
# Célula para aplicar a regra de negócio e filtrar a tabela final

print("Aplicando a regra de negócio para filtrar os aplicativos...")

# calcular o total de proprietários
df_apps_com_tudo['total_proprietarios'] = 1 + df_apps_com_tudo['admin_appsharededitors'].fillna(0).astype(int)

# filtro para selecionar os aplicativos que possuem mais usuários do que proprietários
regra_filtro = df_apps_com_tudo['usuarios_unicos'] > df_apps_com_tudo['total_proprietarios']

# aplica o filtro na tabela final
tabela_final_matheus = df_apps_com_tudo[regra_filtro]

print(f"\nFiltro aplicado! Foram encontrados {len(tabela_final_matheus)} aplicativos que atendem à regra.")
print("\nAbaixo, a tabela final com as informações solicitadas, incluindo o Ambiente:")

# Adicionamos a coluna do ambiente na lista de colunas para exibir
colunas_para_exibir = [
    'admin_displayname_app',
    'admin_displayname_ambiente',
    'admin_appownerdisplayname',
    'total_proprietarios',
    'usuarios_unicos',
    'total_sessoes',
    'ultimo_acesso'
]

# Renomeamos as colunas para a apresentação final
tabela_renomeada = tabela_final_matheus[colunas_para_exibir].rename(columns={  # type: ignore
    'admin_displayname_app': 'Nome do App',
    'admin_displayname_ambiente': 'Ambiente', 
    'admin_appownerdisplayname': 'Proprietário Principal',
    'total_proprietarios': 'Total de Proprietários/Editores',
    'usuarios_unicos': 'Usuários Únicos',
    'total_sessoes': 'Total de Sessões',
    'ultimo_acesso': 'Último Acesso'
})

display(tabela_renomeada)





Aplicando a regra de negócio para filtrar os aplicativos...

✅ Filtro aplicado! Foram encontrados 304 aplicativos que atendem à regra.

Abaixo, a tabela final com as informações solicitadas, incluindo o Ambiente:


,Nome do App,Ambiente,Proprietário Principal,Total de Proprietários/Editores,Usuários Únicos,Total de Sessões,Último Acesso
0,[Licença] - Gerenciamento de Licenças,PRD_IDCER_SOLUTIONS,Priscilla Maria Machado Lopes e Lopes,5,721,2479,2025-06-25 11:49:56
1,Resultado da Avaliação de Integridade (RAI),PRD_RAI_GESTAO_REL_INTEGRIDADE,svc_low_idcer_rai01,2,9,200,2025-06-25 12:01:52
2,Chamados - Estudos Econômicos,NaN,Juan Emilio Duarte Chaparro,4,63,615,2025-06-24 13:34:43
3,Gestão de Avaliações da BRMED,NaN,Vinicius Rodrigues Da Cruz,2,16,320,2025-06-24 12:52:12
4,Chamados - FRPCA,NaN,Juan Emilio Duarte Chaparro,6,238,2623,2025-06-25 12:19:39
...,...,...,...,...,...,...,...
3372,Resultado da Avaliação de Integridade (RAI),NaN,Thalles Cândido Sousa,2,7,293,2024-11-19 13:00:06
3414,Gestão de Avaliações - BRMED,NaN,Vinicius Rodrigues Da Cruz,2,8,50,2024-10-15 16:34:02
3956,Estratégia de Gestão de Talentos - VGC,Ambiente GCEES,Matheus Cota,1,13,18,2024-04-29 15:02:44
3958,Portal de Documentos,NaN,svc_low_idcer_coe02,1,2,4,2024-04-19 17:05:40


In [26]:
try:
    total_de_apps_ambiente = len(df_apps)
    apps_com_algum_uso = len(metricas_de_uso)
    apps_prioritarios = len(tabela_final_matheus)

    tabela_dimensao_macro = pd.DataFrame({
        'Etapa do Funil de Governança': [
            '1. Total de Apps no Ambiente',
            '2. Apps com Registro de Uso (Universo Relevante)',
            '3. Apps de Alto Impacto para Cadastro (Universo Prioritário)'
        ],
        'Quantidade de Aplicativos': [
            total_de_apps_ambiente,
            apps_com_algum_uso,
            apps_prioritarios
        ]
    })

    # Adicionando uma coluna de percentual para dar mais contexto
    tabela_dimensao_macro['% em Relação ao Total'] = \
        (tabela_dimensao_macro['Quantidade de Aplicativos'] / total_de_apps_ambiente * 100).round(1).astype(str) + '%'

    print("Tabela de Dimensão Macro para o Projeto do Catálogo de Soluções")
    display(tabela_dimensao_macro)

except NameError as e:
    print(f"Erro: Uma das tabelas necessárias não foi definida. Verifique a execução das células anteriores.")
    print(f"Detalhe do erro: {e}")


Tabela de Dimensão Macro para o Projeto do Catálogo de Soluções


,Etapa do Funil de Governança,Quantidade de Aplicativos,% em Relação ao Total
0,1. Total de Apps no Ambiente,4036,100.0%
1,2. Apps com Registro de Uso (Universo Relevante),835,20.7%
2,3. Apps de Alto Impacto para Cadastro (Univers...,304,7.5%


In [27]:
# Célula final: Exportando os artefatos da análise de uso para CSV

try:
    # Definindo os nomes dos arquivos
    arquivo_apps_com_metricas = 'tabela_apps_com_metricas_de_uso.csv'
    arquivo_alta_adocao = 'tabela_apps_alta_adocao.csv'
    arquivo_resumo_macro = 'resumo_macro_adocao.csv'

    # Exportando cada DataFrame para um arquivo CSV separado
    df_apps_com_metricas.to_csv(arquivo_apps_com_metricas, index=False, encoding='utf-8-sig')
    tabela_final_matheus.to_csv(arquivo_alta_adocao, index=False, encoding='utf-8-sig')
    tabela_dimensao_macro.to_csv(arquivo_resumo_macro, index=False, encoding='utf-8-sig')

    print("Sucesso! Os três arquivos CSV focados em ADOÇÃO foram exportados:")
    print(f"   1. {arquivo_apps_com_metricas} (Sua base de dados completa para explorar)")
    print(f"   2. {arquivo_alta_adocao} (A lista prioritária da 'missão Matheus')")
    print(f"   3. {arquivo_resumo_macro} (Os números para os KPIs principais)")

except Exception as e:
    print(f"Ocorreu um erro ao exportar os arquivos: {e}")

Sucesso! Os três arquivos CSV focados em ADOÇÃO foram exportados:
   1. tabela_apps_com_metricas_de_uso.csv (Sua base de dados completa para explorar)
   2. tabela_apps_alta_adocao.csv (A lista prioritária da 'missão Matheus')
   3. resumo_macro_adocao.csv (Os números para os KPIs principais)
